In [ ]:
from datasets import load_dataset, Dataset,DatasetDict, load_from_disk
from bigdata_a3_utils import download_all_amazon_reviews, load_compressed_dataset
from pathlib import Path
import polars as pl
import pandas as pd
import itertools
from itertools import islice

In [ ]:
auto_review = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_review_Automotive",
    streaming=True,
    trust_remote_code=True
)

In [ ]:
dataset: Dataset = auto_review["full"]

In [ ]:
batch = list(islice(dataset, 100))
# df = pl.DataFrame(batch)
# df

In [ ]:
full_batch = list(islice(dataset, None))  # islice(..., None) consumes all

In [ ]:
# buffer_size = 100_000
# stream = iter(dataset)

# while True:
#     batch = list(islice(stream, buffer_size))
#     if not batch:
#         break
#     print(batch[0])  # example

In [ ]:
# save_path = Path("data/raw/raw_review_Automotive")
# # auto_review.save_to_disk(str(save_path))

In [ ]:
# loaded_review: DatasetDict = load_from_disk(str(save_path))

In [ ]:
# subset: Dataset = loaded_review["full"]
# subset

In [ ]:
# df = pl.DataFrame(subset[0:9_955_450])
# df = pl.DataFrame(subset[0:19_955_450])

In [ ]:
# df